## Do not really change this, to change the strategies, change strategy.py

### Simple backtesting using fixed parameters:

In [3]:
### Now set as seperate file 'strategies.py'
### This is the printing function for backtrading

import backtrader as bt

class PrintClose(bt.Strategy):

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries to use for analysis
        self.dataclose = self.datas[0].close

    def log(self, txt, dt=None):
        dt = dt 
        date = self.datas[0].datetime.date(0)
        print(f'date: {date}, {txt} {dt}') #Print date and close

    def next(self):
        self.log('Close: ', self.dataclose[0])

In [5]:
### This is the bread and butter, the backtesting running algorithm

import yfinance as yf
import pandas as pd
from IPython.display import display
from strategies import *
from datetime import date, datetime

symbols = ['tsla']#['fb', 'amzn', 'aapl', 'nflx', 'goog']

for i, symbol in enumerate(symbols):
    
    data = bt.feeds.PandasData(dataname=yf.download(symbol, date(date.today().year, 1, 1), datetime.today()))
    
    #Instantiate Cerebro engine
    cerebro = bt.Cerebro()

    #Add data feed to Cerebro
    cerebro.adddata(data)

    # Add strategy to Cerebro
    cerebro.addstrategy(MAcrossover)

    # Default position size - 3 shares
    cerebro.addsizer(bt.sizers.SizerFix, stake=3)

    if __name__ == '__main__':
        # Run Cerebro Engine - pull portfolio value before running for staring funds
        start_portfolio_value = cerebro.broker.getvalue()

        cerebro.run()
        
        #pull portfolio value after running for ending funds
        end_portfolio_value = cerebro.broker.getvalue()
        pnl = end_portfolio_value - start_portfolio_value
        print()
        print(f'Starting Portfolio Value: {start_portfolio_value:2f}')
        print(f'Final Portfolio Value: {end_portfolio_value:2f}')
        print(f'PnL: {pnl:.2f}')

[*********************100%***********************]  1 of 1 completed
2021-02-09, SELL CREATE 849.460022 None
2021-02-10, SELL EXECUTED, 843.64 None
2021-02-18, CLOSE CREATE 787.380005 None
2021-02-19, BUY EXECUTED, 795.00 None
2021-03-22, BUY CREATE 670.000000 None
2021-03-23, BUY EXECUTED, 675.77 None
2021-03-30, CLOSE CREATE 635.619995 None
2021-03-31, SELL EXECUTED, 646.62 None
2021-04-12, BUY CREATE 701.979980 None
2021-04-13, BUY EXECUTED, 712.70 None
2021-04-20, CLOSE CREATE 718.989990 None
2021-04-21, SELL EXECUTED, 704.77 None
2021-05-04, SELL CREATE 673.599976 None
2021-05-05, SELL EXECUTED, 681.06 None
2021-05-12, CLOSE CREATE 589.890015 None
2021-05-13, BUY EXECUTED, 601.54 None
2021-06-03, BUY CREATE 572.840027 None
2021-06-04, BUY EXECUTED, 579.71 None
2021-06-11, CLOSE CREATE 609.890015 None
2021-06-14, SELL EXECUTED, 612.23 None
2021-06-16, SELL CREATE 604.869995 None
2021-06-17, SELL EXECUTED, 601.89 None
2021-06-24, CLOSE CREATE 679.820007 None
2021-06-25, BUY EXECUTED

### Optimising the parameters (e.g. [pfast, pslow]):

In [38]:
### Now for optimising the backtesting w.r.t. the parameters, note we don't want to print every run as there will be many run iterations.

import yfinance as yf
import pandas as pd
from IPython.display import display
from strategies import *
from datetime import date, datetime

symbols = ['goog']#['fb', 'amzn', 'aapl', 'nflx', 'goog']

for i, symbol in enumerate(symbols):
    
    fromdate = datetime(2016,1,1)#date(date.today().year, 1, 1)
    todate = datetime(2017,12,25)#datetime.today()
    data = bt.feeds.PandasData(dataname=yf.download(symbol, fromdate, todate))
    
    # Instantiate Cerebro engine
    cerebro = bt.Cerebro(optreturn=False)

    # Add data feed to Cerebro
    cerebro.adddata(data)

    # We have asked cerebro not to return parameters above, but still need something to optimise, hence we now add an analyser:
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio') 
    # The cerebro strategy to optimise and the range of values the parameters can take:
    cerebro.optstrategy(MAcrossover, pfast=range(5,20), pslow=range(80,100))

    # Default position size - 3 shares
    cerebro.addsizer(bt.sizers.SizerFix, stake=3)

    if __name__ == '__main__':
        # Run Cerebro Engine - pull portfolio value before running for staring funds
        start_portfolio_value = cerebro.broker.getvalue()

        optimised_runs = cerebro.run()
        
        final_results_list = []
        for run in optimised_runs: # a list of lists, each element of the parent list is a set of strategies ([pfast,pslow] combination) 
            for strategy in run:
                # 'Profit and Loss = final value - starting value'
                PnL = strategy.broker.get_value() - start_portfolio_value
                sharpe = strategy.analyzers.sharpe_ratio.get_analysis()
                final_results_list.append([strategy.params.pfast, strategy.params.pslow, PnL, sharpe['sharperatio']])
        
        # sort by third value of 'final_results_list' - sharpe ratio
        sharpe_sorted_reverse = sorted(final_results_list, key=lambda x: x[3], reverse=True)

[*********************100%***********************]  1 of 1 completed


In [46]:
import numpy as np
print('[pfast, pslow, PnL, Sharpe Ratio]: (optimal Sharpe Ratio first ordering)\n')
print(np.array(sharpe_sorted_reverse[:5]))

[pfast, pslow, PnL, Sharpe Ratio]: (optimal Sharpe Ratio first ordering)

[[ 13.          85.         263.16046143   4.70736518]
 [ 12.          88.         240.63024902   4.26173323]
 [ 12.          89.         240.63024902   4.26173323]
 [ 11.          97.         234.24041748   2.55584635]
 [ 17.          80.         262.88964844   1.93025845]]


### Building a stock screener and chart: